# Phenylketonuria predictions with the human metabolic map RECON2

<p>**Authors**: Thierry D.G.A Mondeel, Stefania Astrologo, Ewelina Weglarz-Tomczak & Hans V. Westerhoff <br/>
University of Amsterdam <br/>2016 - 2019
</p>

## Aim of this notebook
In this notebook we will aim to reproduce the inborn error of metabolism phenylketonuria (PKU) on the human metabolic reconstruction. In patients with PKU the enzyme converting phenylalanine into tyrosine (PAH) is not functional. 

We will aim to show that the human metabolic reconstruction model is able to correctly predict that knockout of the PKU gene leads to problems in growth and neurotransmitter productions.

## Setup
We start by setting up the python environment and loading RECON2. This part may take 10 seconds or so. 

In [ ]:
import cobra
from cobra.flux_analysis import pfba
import pandas as pd # for nice tables
pd.set_option('display.max_colwidth', -1)
from utils import show_map
import escher

map_loc = './maps/escher_map_RECON2_AA_metabolism.json' # the escher map used below

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

M = cobra.io.load_json_model('models/recon_2_2_simple_medium.json')
model = M.copy() # this way we can edit the model but leave M unaltered

## Phenylalanine Catabolism Is Genetically Defective in Some People}
<span style="color:red">**Assignment (10 min):**</span> Read the text below and study the images.

The following is taking from Lehninger's principles of biochemistry. 

Given that many amino acids are either neuro-transmitters or precursors or antagonists of neutrotransmitters, genetic defects of amino acid metabolism can cause defective neural development and mental retardation. In most such diseases specific intermediates accumulate. For example, a genetic defect in phenylalanine hydroxylase, the first enzyme in the catabolic pathway for phenylalanine, is responsible for the disease phenylketonuria (PKU), the most common cause of elevated levels of phenylalanine (hyperphenylalaninemia). Phenylalanine hydroxylase (also called phenylalanine-4-monooxygenase) is one of a general class of enzymes called mixed-function oxidases, all of which catalyze simultaneous hydroxylation of a substrate by an oxygen atom of $O_2$ and reduction of the other oxygen atom to $H_2O$. Phenylalanine hydroxylase requires the cofactor tetrahydrobiopterin, which carries electrons from NADH to $O_2$ and becomes oxidized to dihydrobiopterin in the process. It is subsequently reduced by the enzyme dihydrobiopterin reductase in a reaction that requires NADH.

In individuals with PKU, a secondary, normally little-used pathway of phenylalanine metabolism comes into play. In this pathway phenylalanine undergoes transamination with pyruvate to yield phenylpyruvate. Phenylalanine and phenylpyruvate accumulate in the blood and tissues and are excreted in the urine—hence the name “phenylketonuria.” Much of the phenylpyruvate, rather than being excreted as such, is either decarboxylated to phenylacetate or reduced to phenyllactate. Phenylacetate imparts a characteristic odor to the urine, which nurses have traditionally used to detect PKU in infants. The accumulation of phenylalanine or its metabolites in early life impairs normal development of the brain, causing severe mental retardation. This may be caused by excess phenylalanine competing with other amino acids for transport across the blood-brain barrier, resulting in a deficit of required metabolites.

Phenylketonuria was among the first inheritable metabolic defects discovered in humans. When this condition is recognized early in infancy, mental retardation can largely be prevented by rigid dietary control. The diet must supply only enough phenylalanine and tyrosine to meet the needs for protein synthesis. Consumption of protein-rich foods must be curtailed. Natural proteins, such as casein of milk, must first be hydrolyzed and much of the phenylalanine removed to provide an appropriate diet, at least through childhood. Because the artificial sweetener aspartame is a dipeptide of aspartate and the methyl ester of phenylalanine, foods sweetened with aspartame bear warnings addressed to individuals on phenylalanine-controlled diets.

Also see Figure 1 below for a pathway visualization of some more common IEMs.


<img src="images/lehninger-IEMs.png" style="width: 650px;"/>
**Figure 1:** Catabolic pathways for phenylalanine and tyrosine. In humans these amino acids are normally converted to acetoacetyl-CoA and fumarate. Genetic defects in many of these enzymes cause inheritable human diseases (shaded yellow). Source: Lehninger. 


Phenylketonuria can also be caused by a defect in the enzyme that catalyzes the regeneration of tetrahydrobiopterin. The treatment in this case is more complex than restricting the intake of phenylalanine and tyrosine.

Tetrahydrobiopterin is also required for the formation of L-3,4 dihydroxyphenylalanine (L-dopa) and 5-hydroxytryptophan precursors of the neurotransmitters norepinephrine and serotonin, respectively and in phenylketonuria of this type, these precursors must be supplied in the diet. Supplementing the diet with tetrahydrobiopterin itself is ineffective because it is unstable and does not cross the blood-brain barrier.

Also see Figure 2 for the salvage reaction of biopterin.

![biopterin_salvage.png](images/biopterin_salvage.png)
**Figure 2:** Role of tetrahydrobiopterin in the phenylalanine hydroxylase reaction. Source: Lehninger. 

## Preliminary network investigation

Now we will investigate to see if the human metabolic reconstruction implements the network surrounding PAH as described in Lehninger. The image above, is a simplified view of the PAH enzyme and the recycling of the BH4 co-factor.

<span style="color:red">**Assignment (3 min):**</span>
Below, we printed all the reactions phenylalanine engages in. Inspect them (not just their names but their reaction equations).

In [ ]:
df = pd.DataFrame(columns=['ID','Name','Genes','Reaction',]) # empty table

for i,reaction in enumerate(model.metabolites.phe_L_c.reactions):
    df.loc[i] = [reaction.id, reaction.name, reaction.genes, reaction.reaction]

df

<span style="color:red">**Assignment (5 min):**</span> Compare reaction r0399 and the PHETHPTOX2 reaction. What is their relationship? Are they encoded by the same gene or different genes? Check this.

<span style="color:red">**Assignment (5 min):**</span> Investigate the reactions of tetrahydrobiopterin and dihydrobiopterin. How does the recycling occur?

In [ ]:
df = pd.DataFrame(columns=['ID','Name','Genes','Reaction',]) # empty table

for i,reaction in enumerate(model.metabolites.thbpt_c.reactions):
    df.loc[i] = [reaction.id, reaction.name, reaction.genes, reaction.reaction]

df

### A simple representation of the core network surrounding PAH mutation that leads to PKU

![PKU_network](images/PKU_network.png)

<span style="color:red">**Assignment (5 min):**</span> Discuss: is this simplified view of the network surround PAH and the BH4 co-factor correctly describing the RECON 2 network?

To investigate PKU we will ask the model for growth before and after the mutation in the PHETHPTOX2 gene. 

We require that there is always a base level of growth (0.5, arbitrary) so that the model is forced to predict a flux distribution in which growth is occuring.

In [ ]:
model = M.copy()
fvarxns = [model.reactions.r0399, model.reactions.PHETHPTOX2]
cobra.flux_analysis.flux_variability_analysis(model,reaction_list=fvarxns,fraction_of_optimum=0.5)[['minimum','maximum']]

<span style="color:red">**Assignment (2 min):**</span> Are the PHETHPTOX2 and r0399 reactions essential? 
If yes, why? If not, why not?

### Deletions
We will now block r0399 and see what happens.

In [ ]:
model = M.copy()

model.reactions.r0399.upper_bound = 0; model.reactions.r0399.lower_bound = 0;

cobra.flux_analysis.flux_variability_analysis(model,reaction_list=fvarxns,fraction_of_optimum=0.01)[['minimum','maximum']]

<span style="color:red">**Assignment (2 min):**</span> What changed and why?


<span style="color:red">**Assignment (2 min):**</span>
Knockout the gene encoding both the PHETHPTOX2 and r0399 reactions and see what happens if you optimize for growth. What do you expect? Is that what you see?

We already wrote the code that knocks out the gene for you. You should add the FBA simulation and print the flux through the biomass reaction. Try using the summary function like we did in the last notebook.

In [ ]:
model = M.copy() # start fresh

model.reactions.PHETHPTOX2.genes # list the genes

print('PHETHPTOX2 bounds before knockout:',model.reactions.PHETHPTOX2.bounds) # list the gene
print('r0399 bounds before knockout:',model.reactions.r0399.bounds) # list the gene
print()

cobra.manipulation.delete_model_genes(model,['HGNC:8582']) # knock it out

# check that it worked
print('PHETHPTOX2 bounds before knockout:',model.reactions.PHETHPTOX2.bounds)
print('r0399 bounds before knockout:',model.reactions.r0399.bounds) # list the gene
print()

solution = model.optimize()
print('Biomass flux:',solution.f)

## PKU and brain disorders
We generated a so-called escher map on which to visualize our results. This makes it a bit more intuitive to see what is going on.
Take look at the escher map we draw centered around the phenylalanine to tyrosine reaction. 

**Note**: This is only a subset of the reactions in Recon 2. There are many more side branches we could draw here. We tried to emulate here the classical textbook we of representing the pathway.

Can you see from the map how PKU might lead to brain development issues?

In [ ]:
b = show_map([],map_loc)
b.display_in_notebook()

### When the medium includes enough tyrosine and phenylalanine

In the following we will make various alterations to the model and visualize te flux distribution it predicts. We will project the results from flux balance analysis onto the escher map and print in text the flux variability analysis results for a couple key reactions.

Remember that flux balance analysis gives us one flux distribution that achieves the optimal values for the objective reaction. In contrast, flux variability analysis gives us the minimal and maximal flux possible through the reaction while maintaining steady-state balance. 

In [ ]:
model = M.copy()
model.reactions.biomass_reaction.lower_bound = 0.001 # require some biomass flux
model.reactions.EX_tyr_L_e.lower_bound = -1 # more than enough tyrosine

rxnsOfInterest = [model.reactions.get_by_id(x) for x in 
                  ['biomass_reaction','EX_phe_L_e','EX_tyr_L_e','PHETHPTOX2','r0399']]

fvasol = cobra.flux_analysis.flux_variability_analysis(model,reaction_list=rxnsOfInterest,fraction_of_optimum=0)
fvasol[['minimum','maximum']]

<span style="color:red">**Assignment (2 min):**</span>
* The flux variability result shows us the interval for EX_phe_L_e is entirely negative. What does this mean?

Let's visualize the flux distribution on the map.

In [ ]:
fbasol = pfba(model)
b = show_map(fbasol,map_loc)
b.display_in_notebook()

<span style="color:red">**Assignment (2 min):**</span> Do you get why PHETHPTOX2 and r0399 are not active here? Is cofactor recycling active?

## Medium without tyrosine 
Now we reduce the amount of tyrosine in the medium. What will happen?

In [ ]:
model.reactions.EX_tyr_L_e.lower_bound = 0

fbasol = pfba(model)
fvasol = cobra.flux_analysis.flux_variability_analysis(model,reaction_list=rxnsOfInterest,fraction_of_optimum=0)
fvasol[['minimum','maximum']]

b = show_map(fbasol,map_loc)
b.display_in_notebook()

<span style="color:red">**Assignment (2 min):**</span>
* How is it possible that there is still the same biomass reaction flux?
* Why is PHETHPTOX2 not essential here? I.e. why does FVA allow its minimum to be zero?
* What happened to the flux in the recycling pathway of biopterin?

## Can we knock  out one of the two conversion enzymes and get the other one?

In [ ]:
model.reactions.r0399.bounds = (0,0)

fbasol = pfba(model)
fvasol = cobra.flux_analysis.flux_variability_analysis(model,reaction_list=rxnsOfInterest,fraction_of_optimum=0)
fvasol

b = show_map(fbasol,map_loc)
b.display_in_notebook()

<span style="color:red">**Assignment (2 min):**</span> Is PHETHPTOX2 now essential? ie. is it required to carry flux to get growth?

## knockout PAH and its alternative reaction by knocking out the gene

In [ ]:
model.reactions.PHETHPTOX2.genes

<span style="color:red">**Assignment (2 min):**</span> Below we will knock out the genes for the enzyme. See what happens. Does the result make sense?

In [ ]:
### KNOCKOUT THE GENE HERE
g = model.genes.get_by_id('HGNC:8582').knock_out()

fbasol = model.optimize()

## Can we restore life through food?
* before we were missing the backward step from 34hpp (hydroxy phenyl pyruvate): it was not present in the medium by assumption
* Allowing this metabolite in the medium restores growth capability since the metabolic map allows the side branch pathway to run in reverse
* The thermodynamics of this reaction are questionable

In [ ]:
model = M.copy()
model.reactions.EX_34hpp_.lower_bound = -1; # add hydroxyphenylpyruvate to the medium

# perform FVA for some interesting reactions
rxnsOfInterest = [ model.reactions.biomass_reaction, model.reactions.EX_phe_L_e, model.reactions.EX_tyr_L_e, 
                  model.reactions.PHETHPTOX2, model.reactions.r0399, model.reactions.EX_34hpp_]

# knockout the PAH enzyme
model.genes.get_by_id('HGNC:8582').knock_out()

fbasol = pfba(model)
fvasol = cobra.flux_analysis.flux_variability_analysis(model,reaction_list=rxnsOfInterest,fraction_of_optimum=0)
fvasol

b = show_map(fbasol,map_loc)
b.display_in_notebook()

<span style="color:red">**Assignment (5 min):**</span> Even though this may be a simple example what does it teach us about the potential utility of the human metabolic map? 

Can you envision a future in which each individual has his or her genetic mutations mapped onto such a model?
Could we use similar tricks like discussed above to find ways (with food and drugs) around the blockages such mutations generatate?